# ETL Processes
## - Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [1]:
import os
import glob
import psycopg2
import pandas as pd
import numpy as np
from sql_queries import *

In [2]:
conn = psycopg2.connect("host=127.0.0.1 port=5434 dbname=sparkifydb user=postgres password=7890")
cur = conn.cursor()

In [3]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`

## - In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

## - Let's perform ETL on a single song file and load a single record into each table to start.
## - Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
## - Select the first song in this list
## - Read the song file and view the data

In [4]:
song_files = get_files('data/song_data')
#song_files

In [5]:
filepath = song_files[0]
filepath

'c:\\Users\\Serguei\\Dropbox\\1_data_eng\\project_1\\data\\song_data\\A\\A\\A\\TRAAAAW128F429D538.json'

In [6]:
df = pd.read_json(filepath, lines=True)
df.head(5)

,num_songs,artist_id,artist_latitude,artist_longitude,artist_location,artist_name,song_id,title,duration,year
0,1,ARD7TVE1187B99BFB1,NaN,NaN,California - LA,Casual,SOMZWCG12A8C13C480,I Didn't Mean To,218.93179,0


# #1: `songs` Table
## - Extract Data for Songs Table
## - Select columns for song ID, title, artist ID, year, and duration
## - Use `df.values` to select just the values from the dataframe
## - Index to select the first (only) record in the dataframe
## - Convert the array to a list and set it to `song_data`

In [7]:
df.columns


Index(['num_songs', 'artist_id', 'artist_latitude', 'artist_longitude',
       'artist_location', 'artist_name', 'song_id', 'title', 'duration',
       'year'],
      dtype='object')

In [8]:
df[['song_id','title','artist_id','year','duration']].iloc[0]

song_id      SOMZWCG12A8C13C480
title          I Didn't Mean To
artist_id    ARD7TVE1187B99BFB1
year                          0
duration              218.93179
Name: 0, dtype: object

In [9]:
song_data_test  = df[['song_id','title','artist_id','year','duration']].iloc[0].values.tolist()
song_data = list(df[["song_id", "title", "artist_id", "year", "duration"]].values[0])
song_data_test,song_data

(['SOMZWCG12A8C13C480',
  "I Didn't Mean To",
  'ARD7TVE1187B99BFB1',
  0,
  218.93179],
 ['SOMZWCG12A8C13C480',
  "I Didn't Mean To",
  'ARD7TVE1187B99BFB1',
  0,
  218.93179])

# Review data types within song_data

In [10]:
# def checkType(a_list):
#     for element in a_list:
#         if isinstance(element, int):
#             print(element,": ","It's an INTEGER.")
#         if isinstance(element, str):
#             print(element,": ","It's a STRING.")
#         if isinstance(element, float):
#             print(element,": ","It's a FLOAT.")
#         if isinstance(element, list):
#             print(element,": ","It's a LIST.")
#         if isinstance(element, dict):
#             print(element,": ","It's a DICT.")
#         if isinstance(element, tuple):
#             print(element,": ","It's a TUPLE.")
#         if isinstance(element, set):
#             print(element,": ","It's a SET.") 


def checkType(a_list):
    for element in a_list:
        if np.issubdtype(type(element), np.integer):
            print(element,": ","It's an INTEGER.")
        if isinstance(element, str):
            print(element,": ","It's a STRING.")
        if np.issubdtype(type(element), np.float):
            print(element,": ","It's an FLOAT.")
        if isinstance(element, list):
            print(element,": ","It's a LIST.")
        if isinstance(element, dict):
            print(element,": ","It's a DICT.")
        if isinstance(element, tuple):
            print(element,": ","It's a TUPLE.")
        if isinstance(element, set):
            print(element,": ","It's a SET.") 

In [11]:
def getEelementsType(a_list):
    for element in a_list:
        print(f"The list element: {element} has type {type(element)}")

In [12]:
#song_data_test  = df[['song_id','title','artist_id','year','duration']].iloc[0].values.tolist()

getEelementsType(song_data_test)

The list element: SOMZWCG12A8C13C480 has type <class 'str'>
The list element: I Didn't Mean To has type <class 'str'>
The list element: ARD7TVE1187B99BFB1 has type <class 'str'>
The list element: 0 has type <class 'numpy.int64'>
The list element: 218.93179 has type <class 'numpy.float64'>


In [13]:
checkType(song_data_test)

SOMZWCG12A8C13C480 :  It's a STRING.
I Didn't Mean To :  It's a STRING.
ARD7TVE1187B99BFB1 :  It's a STRING.
0 :  It's an INTEGER.
218.93179 :  It's an FLOAT.


In [14]:
#song_data = list(df[["song_id", "title", "artist_id", "year", "duration"]].values[0])

getEelementsType(song_data)

The list element: SOMZWCG12A8C13C480 has type <class 'str'>
The list element: I Didn't Mean To has type <class 'str'>
The list element: ARD7TVE1187B99BFB1 has type <class 'str'>
The list element: 0 has type <class 'int'>
The list element: 218.93179 has type <class 'float'>


In [15]:
checkType(song_data)

SOMZWCG12A8C13C480 :  It's a STRING.
I Didn't Mean To :  It's a STRING.
ARD7TVE1187B99BFB1 :  It's a STRING.
0 :  It's an INTEGER.
218.93179 :  It's an FLOAT.


# Insert Record into Songy Table
## - Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. 
## - Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [16]:
cur.execute(song_table_insert, song_data)
conn.commit()

# Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
# - Select columns for artist ID, name, location, latitude, and longitude
# - Use `df.values` to select just the values from the dataframe
# - Index to select the first (only) record in the dataframe
# - Convert the array to a list and set it to `artist_data`

In [17]:
#['num_songs', 'artist_id', 'artist_latitude', 'artist_longitude',
#    'artist_location', 'artist_name', 'song_id', 'title', 'duration',
#    'year']
# list(df[["song_id", "title", "artist_id", "year", "duration"]].values[0])

artist_data = list(df[["artist_id", "artist_name", "artist_location", "artist_latitude", "artist_longitude"]].values[0])
artist_data

['ARD7TVE1187B99BFB1', 'Casual', 'California - LA', nan, nan]

# Insert Record into Artist Table
## -  Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. 
## - Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [19]:
cur.execute(artist_table_insert, artist_data)
conn.commit()

## Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
## -  In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

## - Let's perform ETL on a single log file and load a single record into each table.
## - Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
## - Select the first log file in this list
## - Read the log file and view the data

<!-- 2018-11-03-events.json:
{"artist":null,"auth":"Logged In","firstName":"Walter","gender":"M","itemInSession":0,"lastName":"Frye","length":null,"level":"free","location":"San Francisco-Oakland-Hayward, CA","method":"GET","page":"Home","registration":1540919166796.0,"sessionId":38,"song":null,"status":200,"ts":1541105830796,"userAgent":"\"Mozilla\/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit\/537.36 (KHTML, like Gecko) Chrome\/36.0.1985.143 Safari\/537.36\"","userId":"39"} -->

In [22]:
log_files = get_files('data/log_data')
# log_files

In [23]:
filepath = log_files[0]

In [24]:
df = pd.read_json(filepath,lines=True)
df.head(2)

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,None,Logged In,Walter,M,0,Frye,NaN,free,"San Francisco-Oakland-Hayward, CA",GET,Home,1540919166796,38,None,200,1541105830796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",39
1,None,Logged In,Kaylee,F,0,Summers,NaN,free,"Phoenix-Mesa-Scottsdale, AZ",GET,Home,1540344794796,139,None,200,1541106106796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8


# #3: `time` Table
## Extract Data for Time Table
## - Filter records by `NextSong` action
## - Convert the `ts` timestamp column to datetime
##   - Hint: the current timestamp is in milliseconds
## - Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
##   - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
## - Specify labels for these columns and set to `column_labels`
## - Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [25]:
#Filter records by NextSong action
#df = df.set_index(['page']).sort_index()
df = df[df.page == 'NextSong']

In [26]:
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
2,Des'ree,Logged In,Kaylee,F,1,Summers,246.30812,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,You Gotta Be,200,1541106106796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
4,Mr Oizo,Logged In,Kaylee,F,3,Summers,144.03873,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Flat 55,200,1541106352796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
5,Tamba Trio,Logged In,Kaylee,F,4,Summers,177.18812,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Quem Quiser Encontrar O Amor,200,1541106496796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
6,The Mars Volta,Logged In,Kaylee,F,5,Summers,380.42077,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Eriatarka,200,1541106673796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
7,Infected Mushroom,Logged In,Kaylee,F,6,Summers,440.26730,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Becoming Insane,200,1541107053796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8


In [28]:
#Convert the ts timestamp column to datetime
timestamp_col = pd.to_datetime(df['ts'],unit="ms")

In [29]:
# Extract the timestamp, hour, day, week of year, month, year, and weekday from the ts column and set time_data to a list containing these values in order
# use pandas' dt attribute to access easily datetimelike properties.


time_data = [df['ts'].values,
timestamp_col.dt.hour.values, 
timestamp_col.dt.day.values, 
timestamp_col.dt.weekofyear.values,
timestamp_col.dt.month.values, 
timestamp_col.dt.year.values,
timestamp_col.dt.weekday.values]

# len(time_data), len(time_data[0])

type(time_data),time_data

<ipython-input-29-c4e069d02b31>:8: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  timestamp_col.dt.weekofyear.values,


(list,
 [array([1541106106796, 1541106352796, 1541106496796, 1541106673796,
         1541107053796, 1541107493796, 1541107734796, 1541108520796,
         1541109125796, 1541109325796, 1541110994796], dtype=int64),
  array([21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 22], dtype=int64),
  array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int64),
  array([44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44], dtype=int64),
  array([11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11], dtype=int64),
  array([2018, 2018, 2018, 2018, 2018, 2018, 2018, 2018, 2018, 2018, 2018],
        dtype=int64),
  array([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3], dtype=int64)])

In [30]:
# Specify labels for these columns and set to column_labels

column_labels = ['start_time','hour', 'day', 'week', 'month', 'year', 'weekday']

# Create a dataframe, time_df, containing the time data for this file by combining column_labels and time_data into a dictionary and converting this into a dataframe

dict_data =  dict(zip(column_labels,time_data))
dict_data

{'start_time': array([1541106106796, 1541106352796, 1541106496796, 1541106673796,
        1541107053796, 1541107493796, 1541107734796, 1541108520796,
        1541109125796, 1541109325796, 1541110994796], dtype=int64),
 'hour': array([21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 22], dtype=int64),
 'day': array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int64),
 'week': array([44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44], dtype=int64),
 'month': array([11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11], dtype=int64),
 'year': array([2018, 2018, 2018, 2018, 2018, 2018, 2018, 2018, 2018, 2018, 2018],
       dtype=int64),
 'weekday': array([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3], dtype=int64)}

In [31]:
time_df = pd.DataFrame(dict_data)
time_df.head()

,start_time,hour,day,week,month,year,weekday
0,1541106106796,21,1,44,11,2018,3
1,1541106352796,21,1,44,11,2018,3
2,1541106496796,21,1,44,11,2018,3
3,1541106673796,21,1,44,11,2018,3
4,1541107053796,21,1,44,11,2018,3


# Insert Records into Time Table
## - Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. 
## - Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [33]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

## Run `test.ipynb` to see if you've successfully added records to this table.

# #4: `users` Table
## - Extract Data for Users Table
## - Select columns for user ID, first name, last name, gender and level and set to `user_df`



In [34]:
user_df = df[['userId', 'firstName', 'lastName', 'gender', 'level']]

# Insert Records into Users Table
## -  Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. 
## -  Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [35]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

In [36]:
df.head(2)

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
2,Des'ree,Logged In,Kaylee,F,1,Summers,246.30812,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,You Gotta Be,200,1541106106796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
4,Mr Oizo,Logged In,Kaylee,F,3,Summers,144.03873,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Flat 55,200,1541106352796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8


## Run `test.ipynb` to see if you've successfully added records to this table.

# #5: `songplays` Table
## - Extract Data and Songplays Table
## - This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table.
## - Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.

## - Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.

## - Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`


# Insert Records into Songplays Table
## - Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. 
## Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [37]:
for index, row in df.iterrows():
    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None
    # insert songplay record
    songplay_data = [row.ts, row.userId, row.level, songid, artistid,
                    row.sessionId, row.location, row.userAgent]
    cur.execute(songplay_table_insert,songplay_data )
    conn.commit()

## Run `test.ipynb` to see if you've successfully added records to this table.

## Close Connection to Sparkify Database

In [38]:
conn.close()

## Implement `etl.py`
## - Use these steps to implement `etl.py`.